In [990]:
import pandas as pd
from Bio import Entrez, Medline
import glob
import requests
import re
import os
import sys
from collections import defaultdict
import tarfile
from bs4 import BeautifulSoup
import numpy as np
from pathlib import Path
import spacy
from orkg import ORKG 

In [ ]:
Entrez.api_key = ""
Entrez.email = ""

# 1. Creating folders

In [ ]:
def create_folders(folder_list):
    for i in folder_list:
        if not os.path.exists("articles/"+i):
            os.makedirs("articles/"+i)

In [ ]:
folder_list = ["zipped_documents", "unzipped_documents", "xml_files", "dataframes"]
create_folders(folder_list)

# 2. Retrieve FTP-links and download the fulltexts

In [ ]:
def retrieving_ftp_links(query, amount):
    pubmed_id_list = []
    handle = Entrez.esearch(db="pmc", term=query, retmax=amount)
    record = Entrez.read(handle)
    pubmed_id_list.append(record["IdList"])

    pmc_list = [item for sublist in pubmed_id_list for item in sublist]
    
    liste_ftp_links = []
    
    for i in pmc_list:
        page2 = requests.get('https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id='+i)
        page = requests.get('https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id='+i).content
        soup = BeautifulSoup(page, 'html.parser')
        if "</error>" in soup:
            continue
        else:
        
            for link in soup.find_all('link'):
                if link.get('href').endswith('tar.gz'):
                    ftp_link = link.get('href')
                    liste_ftp_links.append(ftp_link[6:])
        
    return liste_ftp_links

In [ ]:
ftp_lst = retrieving_ftp_links("rabies hosts","10")

In [ ]:
def downloading_fulltext(liste_ftp_links):
    for i in liste_ftp_links:
        path = "articles/zipped_documents/" + i[-17:]
        response = requests.get("http://"+ i, stream=True)
        if response.status_code == 200:
            with open(path, 'wb') as f:
                f.write(response.raw.read())

In [ ]:
downloading_fulltext(ftp_lst)

In [ ]:
liste_zipped_files = glob.glob("articles/zipped_documents/*")

# 3. Extract the downloaded zip-files and shift the xml-files in the one folder

In [ ]:
def extract_files(liste_zipped_files):
    for i in liste_zipped_files:
        if i.endswith("tar.gz"):
            tar = tarfile.open(i, "r:gz")
            tar.extractall("articles/unzipped_documents")
            tar.close()
        elif i.endswith("tar"):
            tar = tarfile.open(i, "r:")
            tar.extractall()
            tar.close()

In [ ]:
extract_files(liste_zipped_files)

In [ ]:
def rename_files():
    liste_xml = glob.glob("articles/unzipped_documents/*/*.nxml")

    for i in liste_xml:
        m = re.search("PMC\d+", i).group(0)
        #print(m)

        os.rename(i, "articles/xml_files/" + m + '.nxml')

In [ ]:
rename_files()

# 4. Create dataframe with the Metadata from Entrez and the fulltexts within the xml-files

In [ ]:
def get_nxml_paths():
    liste_zipped_files = glob.glob("articles/xml_files/*.nxml")

    return liste_zipped_files

In [ ]:
def change_type_df_column(df):
    import numpy as np
    col_list = ["PMC"]
    for i in col_list:
        bytelist = df[i].tolist()
        stringlist = [x[0].decode('utf-8') for x in bytelist]
        df[i] = stringlist

    col_list_2 = ["TI" ,"FAU","AID","AUID", "JT", "DP"]

    for i in col_list_2:
        liste = []
        bytelist = df[i].tolist()

        for j in bytelist:
            if type(j) != float:
                stringlist = [x.decode('utf-8') for x in j]
                liste.append(stringlist)
            else:
                liste.append([])
        df[i] = liste
    return df

In [ ]:
def retrieve_pmc_list():
    liste_zipped_files = glob.glob("articles/xml_files/*.nxml")
    flat_list = []
    for i in liste_zipped_files:
        i = i.replace(".nxml","")
        flat_list.append(i[-10:])

    handle = Entrez.efetch(db="pmc", id=flat_list, rettype="medline", retmode="json")
    records = Medline.parse(handle)
    data_list = []
    for record in records:
        data_list.append(record)
    df = pd.DataFrame(data_list)
    df.columns = df.columns.astype(str)



    df_filtered = df[["PMC","TI" ,"FAU","AID","AUID", "JT", "DP"]]

    df_filtered = change_type_df_column(df_filtered)


    return df_filtered

In [1159]:
def extract_body_content(df_metadata):
    liste_zipped_files =  get_nxml_paths()
    liste_text = []
    liste_name = []
    for i in liste_zipped_files:
        temp_text_list = []
        with open(i, "r", encoding="utf-8") as f:
            article = f.read()
            f.close()
        soup = BeautifulSoup(article, "lxml")
        for text in soup.find_all("p"):
            temp_text_list.append(text)

        name_str = "PMC"+soup.find("article-id", {"pub-id-type": "pmc"}).text
        liste_name.append(name_str)
        liste_text.append(temp_text_list)
        
    df = pd.DataFrame()
    df["PMCID"] = liste_name
    df["text"] = liste_text
   
    df = df.merge(df_metadata, left_on="PMCID", right_on="PMC")
    df["text"] = df["text"].apply(lambda x: re.sub(r'<.*?>', '', str(x)))
    return df

# 5. Use Named Entity Recognizer from the trained model

In [1160]:
output_dir = Path('models/')

In [1161]:
def locate_entities(df):
    wanted_entities = ["INCIDENCE", "GPE", "VIRUS", "HOST", "LOC", "DATE_2", "MORTALITY", "RVALUE", "HOSPITALIZATION"]
    df["entities"] = ""
    entity_list = []
    for i in df["text"]:
        doc = nlp(i)
        entity_list.append([(ent.text, ent.label_) for ent in doc.ents if ent.label_ in wanted_entities])
    df["entities"] = entity_list
    df = df.drop("text", axis=1)
    
    return df

## The values that are used have a specific P-value, if they already exist in the ORKG

### These values have to be looked up on https://www.orkg.org/orkg/properties

#### P values for the labels:
- GPE, LOC: P15698
- VIRUS: P43044
- HOST: P43040
- INCIDENCE: P43045
- MORTALITY: P43041
- RVALUE: P41810
- HOSPITALIZATION: P43046
- DATE_2: P9069

# 6. Create Dataframe with the data that will be implemented into the ORKG

In [1162]:
def create_entity_columns(df):
    label_liste=[]
    for i in df["entities"]:
        for j in i:
            label_liste.append((j[1]))
    label_liste = list(set(label_liste))
    for i in label_liste:
        df[i] = ""
    return(df)

In [1163]:
def transform_dataframe(df):
    doi_list = []
    month_list = []
    year_list = []
    for i in df["AID"]:
        temp_doi_list = []
        doi_list.append(i[0].replace("[doi]", ""))
    for i in df["DP"]:
        year_list.append(i[0][:4])
        month_list.append(i[0][5:])
    df["AID"] = doi_list
    df["year"] = year_list
    df["month"] = month_list
    df = df.drop("DP", axis=1)
    df = df.drop("PMCID", axis=1)
    
    return df

In [1164]:
df_metadata = retrieve_pmc_list()
df = extract_body_content(df_metadata)
df = df.drop("PMC", axis=1)
df = locate_entities(df)
df = replace_labels_with_pvalues(df)
df = clean_dataframe(df)

/tmp/ipykernel_15334/2440220305.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i] = stringlist
/tmp/ipykernel_15334/2440220305.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i] = liste


In [1165]:
def distribute_entities(df):
    final_GPE = []
    final_MORT = []
    final_RVALUE = []
    final_VIRUS =[]
    final_HOST = []
    final_INCIDENCE = []
    final_HOSPITALIZATION = []
    final_DATE_2 = []
    for i in df["entities"]:
        GPE = []
        MORT = []
        RVALUE = []
        VIRUS = []
        HOST = []
        HOSPITALIZATION = []
        INCIDENCE = []
        DATE_2 = []
        for j in i:
            if j[1] == "GPE" or j[1] == "LOC":
                if j[0].startswith("9"):
                    continue
                else:
                    GPE.append(j[0]) 
                
        for j in i:
            
            if j[1] == "MORTALITY":
                MORT.append(j[0])
               
        
        for j in i:
            
            if j[1] == "RVALUE":
                RVALUE.append(j[0])
            
            
        for j in i:
            
            if j[1] == "VIRUS":
                VIRUS.append(j[0])
            
        
        for j in i:
            
            if j[1] == "HOST":
                HOST.append(j[0])
            
            
        for j in i: 
            
            if j[1] == "INCIDENCE":
                INCIDENCE.append(j[0])
            
            
        for j in i:    
            
            if j[1] == "HOSPITALIZATION":
                HOSPITALIZATION.append(j[0])
            
            
        for j in i:
            
            if j[1] == "DATE_2":
                DATE_2.append(j[0])
        
        final_GPE.append(list(set(GPE)))
        final_HOSPITALIZATION.append(list(set(HOSPITALIZATION)))
        final_INCIDENCE.append(list(set(INCIDENCE)))        
        final_HOST.append(list(set(HOST)))       
        final_VIRUS.append(list(set(VIRUS)))
        final_RVALUE.append(list(set(RVALUE)))
        final_DATE_2.append(list(set(DATE_2)))
        final_MORT.append(list(set(MORT)))


    df["GPE"] = final_GPE
    df["MORTALITY"] = final_MORT
    df["RVALUE"] = final_RVALUE
    df["VIRUS"] = final_VIRUS
    df["HOST"] = final_HOST
    df["INCIDENCE"] = final_INCIDENCE
    df["HOSPITALIZATION"] = final_HOSPITALIZATION
    df["DATE_2"] = final_DATE_2
    
    return df

In [1166]:
df = distribute_entities(df)

In [1167]:
df

,TI,FAU,AID,AUID,JT,entities,ents,year,month,GPE,MORTALITY,RVALUE,VIRUS,HOST,INCIDENCE,HOSPITALIZATION,DATE_2
0,[Double-antigen sandwich ELISA based on chimer...,"[Freitas, Natália Erdens Maron, Santos, Emily ...",10.1371/journal.pntd.0010290,"[ORCID: https://orcid.org/0000-0001-6286-1883,...",[PLoS Neglected Tropical Diseases],"[(98.7%, GPE), (99.5%, MORTALITY), (74.4%, MOR...","[(98.7%, P15698), (99.5%, P43041), (74.4%, P43...",2022,Mar,"[Europe, Brazil, US, Denmark, Americas, Austri...","[74.4%, 100%, 99.5%]",[],"[HIV-1/2, HEMOBA, HIV, human immunodeficiency ...",[mammalian],[43],[],[]
1,[Alphacoronavirus in a Daubenton’s Myotis Bat ...,"[Lwande, Olivia Wesula, Thalin, Therese, de Jo...",10.3390/v14030556,"[ORCID: https://orcid.org/0000-0001-5350-4219,...",[Viruses],"[(COVID-19, VIRUS), (Sweden, GPE), (Myotis dau...","[(COVID-19, P43044), (Sweden, P15698), (Myotis...",2022,Mar,"[Russia, CA, Northern Europe, A.S., Germany, C...","[9.6, 6.3%]",[],"[MERS-CoV, coronavirus, COVID-19, Miniopterus,...","[Myotis mystacinus, Myotis daubentonii, cats, ...",[],[6.5],"[2014–2018, 13–24 July 2020, 2002, 2012]"
2,[Transduction of HEK293 Cells with BacMam Bacu...,"[Puente-Massaguer, Eduard, Cajamarca-Berrezuet...",10.3390/v14030636,"[ORCID: https://orcid.org/0000-0002-2816-7051,...",[Viruses],"[(Mount Sinai, GPE), (New York, GPE), (NY 1002...","[(Mount Sinai, P15698), (New York, P15698), (N...",2022,Mar,"[Santa Clara, Darmstadt, Grand Island, Gatan, ...","[1.1%, 50%]",[],"[HIV-1, human immunodeficiency virus, COVID-19]",[],"[14.4 ± 0.2 RFU, 1.1 ± 0.1 ×]",[],[]
3,[The Importance of Accurate Host Species Ident...,"[De Benedictis, Paola, Leopardi, Stefania, Mar...",10.3390/v14030492,"[ORCID: https://orcid.org/0000-0001-6760-1933,...",[Viruses],"[(rabies, VIRUS), (rabies, VIRUS), (Africa, LO...","[(rabies, P43044), (rabies, P43044), (Africa, ...",2022,Mar,"[Iran, China, Lycaon, Brazil, Canidae, Turkey,...","[2.9% to 4.1%, 14.5%, 17.4%, 2.9%]",[],"[COVID-19, RABV, Rabies, SARS-CoV-2, rabies]","[jackal, monkey, jackals, domestic dogs, mongo...",[],[],[]
4,[The Role of Nucleoprotein in Immunity to Huma...,"[Šantak, Maja, Matić, Zrinka]",10.3390/v14030521,[ORCID: https://orcid.org/0000-0003-1489-1003],[Viruses],"[(rabies, VIRUS), (Ebola virus, VIRUS), (rabie...","[(rabies, P43044), (Ebola virus, P43044), (rab...",2022,Mar,"[Zaire, Ankara, Fcγ, Tai Forest virus, Europea...",[78%],[],"[avian H5N1viruses, ADCC, Ebola virus, influen...",[basics],"[11.8 to 15.8 per, 10–11 monomers]",[],"[2020, 1 February 2022, September 2020, 21 Dec..."
5,[Development of Real-Time Molecular Assays for...,"[Faye, Martin, Seye, Thiané, Patel, Pranav, Di...",10.3390/microorganisms10030550,"[ORCID: https://orcid.org/0000-0003-3207-6344,...",[Microorganisms],"[(Africa, LOC), (Africa, LOC), (South Africa, ...","[(Africa, P15698), (Africa, P15698), (South Af...",2022,Mar,"[Darmstadt, West Nile, Heiden, Foster City, Gr...","[100%, 3%, 0%]",[0.002 and 0.001],[Rift Valley Fever],[],[],[],"[1 November 2021, January to December 2019, 15..."
6,[Special Issue: “Viral Genetic Diversity”.],"[Heraud, Jean-Michel, Lavergne, Anne, Njouom, ...",10.3390/v14030570,[ORCID: https://orcid.org/0000-0003-1107-0859],[Viruses],"[(SARS-CoV-2, VIRUS), (SARS-CoV-2, VIRUS), (SA...","[(SARS-CoV-2, P43044), (SARS-CoV-2, P43044), (...",2022,Mar,"[South America, Africa, Taiwan]",[],[],[SARS-CoV-2],[],[],[],[]
7,[Porcine Circoviruses and Herpesviruses Are Pr...,"[Auer, Angelika, Schweitzer, Lea, Kübber-Heiss...",10.3390/pathogens11030305,"[ORCID: https://orcid.org/0000-0003-2852-5047,...",[Pathogens],"[(fall 2019, DATE_2), (herpesvirus, VIRUS), (2...","[(fall 2019, P9069), (herpesvirus, P43044), (2...",2022,Mar,"[Ireland, Europe, Italy, China, L.S., C.R., Qi...","[17%, 74%, 6.7%, 23%, 29%, 4.4%, 54%, 45%, 23%...",[],"[PLHV-2, PLHV-1, rabies, herpesvirus]",[],"[6%, 4%]",[95],"[2020, 2019, 2007 to 2009, fall 2019, 2016, 2004]"
8,[Onset of Oral Lichenoid Lesions and Oral Lich...,"[Hertel, Moritz, Schmidt-Wes

# Create json-format of the dataframe that is ORKG conform and push it

In [ ]:
orkg = ORKG(host="https://www.orkg.org/orkg/", creds=('<username>', '<password>')) # create the connector to the ORKG

In [1178]:
def prepare_data_push_data(df, length):
    
    orkg_dict = {}
    #adding authors
    authors_list = []
    for i,j in test_df.iterrows():
        temp_authors_list = []
        for k in j["FAU"]:
            temp_authors_list.append(k)

        authors_list.append(temp_authors_list)

    list_of_authors = []
    for i in authors_list[length]:
        orkg_dict = {}
        orkg_dict["label"] = i
        list_of_authors.append(orkg_dict)

    gpe_list = []
    for i in df["GPE"][length]:
        if len(i) > 2:
            gpe_dict = {}
            gpe_dict["text"] = i
            gpe_list.append(gpe_dict)

    vir_list = []
    for i in df["VIRUS"][length]:
        vir_dict = {}
        vir_dict["text"] = i
        vir_list.append(vir_dict)

    host_list = []
    for i in df["HOST"][length]:
        host_dict = {}
        host_dict["text"] = i
        host_list.append(host_dict)

    inc_list = []
    for i in df["INCIDENCE"][length]:
        inc_dict = {}
        inc_dict["text"] = i
        inc_list.append(inc_dict)

    mort_list = []
    for i in df["MORTALITY"][length]:
        mort_dict = {}
        mort_dict["text"] = i
        mort_list.append(mort_dict)

    rval_list = []
    for i in df["RVALUE"][length]:
        rval_dict = {}
        rval_dict["text"] = i
        rval_list.append(rval_dict)        

    hospi_list = []
    for i in df["HOSPITALIZATION"][length]:
        hospi_dict = {}
        hospi_dict["text"] = i
        hospi_list.append(hospi_dict)

    date_list = []
    for i in df["DATE_2"][length]:
        date_dict = {}
        date_dict["text"] = i
        date_list.append(date_dict)


    #cleaning and set variable for title
    title = str(test_df["TI"][length]).replace("[","").replace("]","").replace("'","")
    #cleaning and set variable for journal-title
    journal = str(test_df["JT"][length]).replace("[","").replace("]","").replace("'","")
    month = test_df["month"][length]
    year = test_df["year"][length]
    #set variable for doi
    doi = test_df["AID"][length]

    #append variables to dict
    orkg_dict = {}
    pvalues = {}
    contri_dict = {}
    final_dict = {}
    pvalue_dict = {}
    orkg_dict["title"] = title
    orkg_dict["doi"] = doi
    orkg_dict["authors"] = list_of_authors
    orkg_dict["publicationMonth"] = ""
    orkg_dict["publicationYear"] = year
    orkg_dict["publishedIn"] = journal
    orkg_dict["researchField"] = "R57"



    pvalue_dict["label"] =  "Hosts for rabies"
    pvalue_dict["class"] = "Problem"
    pvalue_dict["values"] = {}

    pvalues["P32"] = [pvalue_dict]
    pvalues["P9069"] = date_list
    pvalues["P15698"] = gpe_list
    pvalues["P41810"] = rval_list
    pvalues["P43040"] = host_list
    pvalues["P43041"] = mort_list
    pvalues["P43044"] = vir_list
    pvalues["P43045"] = inc_list
    pvalues["P43046"] = hospi_list


    contri_dict["name"] = "Contribution 1"
    contri_dict["values"] = pvalues

    orkg_dict["contributions"] = [contri_dict]

    final_dict["predicates"] = []
    final_dict["paper"] = orkg_dict 

    paper = final_dict
    orkg.papers.add(params=paper)

In [1179]:
for i in range(10):
    prepare_data_push_data(df,i)

{'predicates': [], 'paper': {'title': 'Double-antigen sandwich ELISA based on chimeric antigens for detection of', 'doi': '10.1371/journal.pntd.0010290 ', 'authors': [{'label': 'Freitas, Natália Erdens Maron'}, {'label': 'Santos, Emily Ferreira'}, {'label': 'Leony, Leonardo Maia'}, {'label': 'Silva, Ângelo Antônio Oliveira'}, {'label': 'Daltro, Ramona Tavares'}, {'label': 'Vasconcelos, Larissa de Carvalho Medrado'}, {'label': 'Duarte, Gabriela Agra'}, {'label': 'Oliveira da Mota, Cristiane'}, {'label': 'Silva, Edimilson Domingos'}, {'label': 'Celedon, Paola Alejandra Fiorani'}, {'label': 'Zanchin, Nilson Ivo Tonin'}, {'label': 'Santos, Fred Luciano Neves'}], 'publicationMonth': '', 'publicationYear': '2022', 'publishedIn': 'PLoS Neglected Tropical Diseases', 'researchField': 'R57', 'contributions': [{'name': 'Contribution 1', 'values': {'P32': [{'label': 'Hosts for rabies', 'class': 'Problem', 'values': {}}], 'P9069': [], 'P15698': [{'text': 'Europe'}, {'text': 'Brazil'}, {'text': 'Den

{'predicates': [], 'paper': {'title': 'Special Issue: “Viral Genetic Diversity”.', 'doi': '10.3390/v14030570 ', 'authors': [{'label': 'Heraud, Jean-Michel'}, {'label': 'Lavergne, Anne'}, {'label': 'Njouom, Richard'}], 'publicationMonth': '', 'publicationYear': '2022', 'publishedIn': 'Viruses', 'researchField': 'R57', 'contributions': [{'name': 'Contribution 1', 'values': {'P32': [{'label': 'Hosts for rabies', 'class': 'Problem', 'values': {}}], 'P9069': [], 'P15698': [{'text': 'South America'}, {'text': 'Africa'}, {'text': 'Taiwan'}], 'P41810': [], 'P43040': [], 'P43041': [], 'P43044': [{'text': 'SARS-CoV-2'}], 'P43045': [], 'P43046': []}}]}}
{'predicates': [], 'paper': {'title': 'Porcine Circoviruses and Herpesviruses Are Prevalent in an Austrian Game', 'doi': '10.3390/pathogens11030305 ', 'authors': [{'label': 'Auer, Angelika'}, {'label': 'Schweitzer, Lea'}, {'label': 'Kübber-Heiss, Anna'}, {'label': 'Posautz, Annika'}, {'label': 'Dimmel, Katharina'}, {'label': 'Seitz, Kerstin'}, {'l

In [1155]:
paper

{'predicates': [],
 'paper': {'title': 'Double-antigen sandwich ELISA based on chimeric antigens for detection of',
  'doi': '10.1371/journal.pntd.0010290 ',
  'authors': [{'label': 'Freitas, Natália Erdens Maron'},
   {'label': 'Santos, Emily Ferreira'},
   {'label': 'Leony, Leonardo Maia'},
   {'label': 'Silva, Ângelo Antônio Oliveira'},
   {'label': 'Daltro, Ramona Tavares'},
   {'label': 'Vasconcelos, Larissa de Carvalho Medrado'},
   {'label': 'Duarte, Gabriela Agra'},
   {'label': 'Oliveira da Mota, Cristiane'},
   {'label': 'Silva, Edimilson Domingos'},
   {'label': 'Celedon, Paola Alejandra Fiorani'},
   {'label': 'Zanchin, Nilson Ivo Tonin'},
   {'label': 'Santos, Fred Luciano Neves'}],
  'publicationMonth': '',
  'publicationYear': '2022',
  'publishedIn': 'PLoS Neglected Tropical Diseases',
  'researchField': 'R57',
  'contributions': [{'name': 'Contribution 1',
    'values': {'P32': [{'label': 'Hosts for rabies',
       'class': 'Problem',
       'values': {}}],
     'P9069

In [1156]:
from orkg import ORKG # import base class from package

orkg = ORKG(host="https://www.orkg.org/orkg/", creds=('joshua.thos@smail.th-koeln.de', 'Kenny1602')) # create the connector to the ORKG